# Settings

In [ ]:
# Läs settings
import json
from pathlib import Path
import os

#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO.json"
settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

#print(settings)
print(json.dumps(settings, indent=4))

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
powerlines_folder = settings["powerlines_folder"]
domains_folder = settings["domains_folder"]
scandate_file = settings["scandate_file"]
working_gdb_template = settings["working_gdb_template"]
results_gdb_template = settings["results_gdb_template"]
cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
RBX_shape_folder = os.path.join(local_dir, f"RBX_shp_{run_ID}")
working_gdb = os.path.join(local_dir, f"working_{run_ID}.gdb")
results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")
LG_polygons = settings["LG_polygons"]
station_polygons = settings["station_polygons"]
module_path = settings["modules"]

sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")
print("körning av cell klar")

# Import modules

In [ ]:
import sys
import importlib

# Den absoluta sökvägen till modules
print(module_path)  # Kontrollera att sökvägen är korrekt
if module_path not in sys.path:
    sys.path.append(module_path)

# Importera utils
import utils
importlib.reload(utils)

# test om utils är laddat
utils.is_utils_loaded()


# Skapa lokala kataloger och kopiera template-gdb:er

In [ ]:
from pathlib import Path
import os
import shutil


# Create empty folder for shapefiles
if os.path.exists(RBX_shape_folder):
    print(f"{RBX_shape_folder} finns redan")
else:
    os.makedirs(RBX_shape_folder)

# Create working gdb as copy of template
if os.path.exists(working_gdb):
    print(f"{working_gdb} finns redan")
else: 
    shutil.copytree(working_gdb_template, working_gdb)

# Create results gdb as copy of template
if os.path.exists(results_gdb):
    print(f"{results_gdb} finns redan")
else: 
    shutil.copytree(results_gdb_template, results_gdb)
print("körning av cell klar")

# Merge SKB text files for all blocks of a powerline
Kör blockskript i respektive servermapp för att ha den absolut senaste uppdaterade.

In [ ]:
import os
import sys
import glob
import fileinput
from pathlib import Path
#sys.path.insert(1, r"C:\Users\semnve\Documents\Visual Studio Code\30047770-SVK-Analys-av-laserdata")
#import SKB
#import SVK
import pandas as pd

def combine_blocks(row):
    LG = row["LG"]
    line = row["line"]
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    print(line_dir)
    
    print(f"Doing {LG}_{line}")
    block_dir = os.path.join(line_dir, "kantträd", "block")
    print(block_dir)
    combined_blocks_path = os.path.join(line_dir, "kantträd", "SKB_raw.txt")
    print(combined_blocks_path)
    
    # Merge files for all blocks into one
    #merge_blocks(block_dir, "*.txt", combined_blocks_path)
    utils.merge_blocks(block_dir, "*.txt", combined_blocks_path)

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(combine_blocks, axis=1)
print("körning av cell klar")

# Calculate distances to wire for trees

In [ ]:
import sys
import os, time, arcpy
from time import gmtime, strftime
from pathlib import Path
import pandas as pd
#import SKB

# Featuredataset för kantträdspunkter innehållande X, Y och markens Z-värde under trädet:
SKB_XYmZ_location = os.path.join(working_gdb, 'SKB_XYmZ')
#print(f"SKB_XYmZ_location: {SKB_XYmZ_location}")
# Featuredataset för kantträdspunkter innehållande X, Y och Z-värde för trädkronans topp:
SKB_XYZ_location = os.path.join(working_gdb, 'SKB_XYZ')
#print(f"SKB_XYZ_location: {SKB_XYZ_location}")

# Featuredataset för faslinor, som i ett tidigare steg lästs in från DGN-filerna (Wires.ipynb):
#wires_fd_1 = os.path.join(gdb, 'Wires_2041205')
wires_fd_1 = wires_gdb
print(f"wires_fd_1: {wires_fd_1}")
# wires_fd_2 = os.path.join(gdb, 'Wires_incomplete') # Troligen en rest från när vi hade dåliga punktmoln som orsakade luckor i faslinorna
wires_suffix = '_fas'
search_radius = 100 # How far from the point should Near3D search for wires
SKB_template = os.path.join(working_gdb, 'SKB_XYZ', 'SKB_template') # Detta bör vara en mall för featureklassen med resultatet som ska skapas
print(f"SKB_template: {SKB_template}")
atgardsar = 2024 # TODO: BÖR INTE VARA MED LÄNGRE, KOLLA SÅ ATT DET INTE ANVÄNDS LÄNGRE NER I SKRIPTET
leverantor = 'SWECO'
insamlingsmetod = 20
matosakerhet_p = 1000
matosakerhet_h = 1000
arcpy.env.overwriteOutput = True

def sweref99TM():
    wkt = 'PROJCS["SWEREF99_TM",GEOGCS["GCS_SWEREF99",DATUM["D_SWEREF99",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["Meter",1]]'
    spatialref = arcpy.SpatialReference()
    spatialref.loadFromString(wkt)
    return(spatialref)


def compute_SKB_distances(row):
    LG = row["LG"] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line = row["line"] # TODO: Hänvisa till kolumnnamn i styrfilen istället för index
    line_dir = Path(powerlines_folder) / LG / f"line_{line}"
    SKB_dir = line_dir / "kantträd" 
    
    arcpy.env.overwriteOutput = True
    print(f"Starting with {LG}_{line}")
    
    # Namn på featureklasser som ska skapas i featuredataset som definierats i början av skriptet:
    SKB_XYmZ = f"{LG}_{line}_SKB_XYmZ"
    SKB_XYZ = f"{LG}_{line}_SKB_XYZ"
    akuta_trad = f"{LG}_{line}_akuta"
    ej_akuta_trad = f"{LG}_{line}_ej_akuta"
    
    # Sökvägar till featureklasserna:
    SKB_XYmZ_path = os.path.join(SKB_XYmZ_location, SKB_XYmZ)
    SKB_XYZ_path = os.path.join(SKB_XYZ_location, SKB_XYZ)
    akuta_trad_path = os.path.join(SKB_XYZ_location, akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta
    ej_akuta_trad_path = os.path.join(SKB_XYZ_location, ej_akuta_trad) # Används ej? Inaktuellt eftersom vi inte längre delar upp kantträd i akuta och ej akuta

    wires_1 = os.path.join(wires_fd_1, f"{LG}_{line}{wires_suffix}")
    # wires_2 = os.path.join(wires_fd_2, f"{LG}_{line}{wires_suffix}") # REST SOM INTE LÄNGRE SKA VARA MED? DEN FANNS NÄR VI HADE DÅLIGA PUNKTMOLN SOM ORSAKADE LUCKOR I FASLINORNA
    
    if arcpy.Exists(wires_1):
        wires = wires_1
        print(f"{LG}_{line} faslinor i Wires")
    #elif arcpy.Exists(wires_2):
    #    wires = wires_2
    #    print(f"{LG}_{line} faslinor i Wires_incomplete")
    else:
        print(f"Inga faslinor för {LG}_{line}")
        return
    
    # Create an fc for X,Y,Z,dZ,mZ with mZ as Z coordinate
    # Z är trädtoppens Z-värde i RH2000 dvs höjd över havet
    # dZ är skillanden mellan markens Z-värde under trädet och trädkronans Z-värde
    # mZ är markens Z-värde under trädet
    # Output från TerraScan innehåller X, Y, Z och dZ.
    SKB_file_in = os.path.join(SKB_dir, f"SKB_raw.txt")
    #SKB.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    utils.create_SKB_XYmZ(SKB_file_in, SKB_XYmZ_location, SKB_XYmZ, sr)
    print(f'{LG}_{line}_SKB_XYmZ created')

    # Call arcpy function Near3D to calculate distance from mZ to wire, 
    # on SKB_XYmZ
    #SKB.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    utils.dist_mZ_wire(SKB_XYmZ_location, SKB_XYmZ, wires, search_radius)
    print(f'dist_mZ_wire calculated for {LG}_{line}')
    print(f'{strftime("%H:%M:%S", gmtime())}')

    # Create feature class for SKB where point XYZ is the XYZ for the tree tops
    #SKB.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)
    utils.create_SKB_XYZ(SKB_XYZ_location, SKB_XYZ, SKB_template, sr)

    # Fill SKB_XYZ with values from SKB_XYmZ and set Z coordinate to Z (instead of mZ)
    LG_nr = int(LG[-3:])
    #SKB.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod,
    #                    matosakerhet_p, matosakerhet_h)
    #populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, atgardsar, leverantor, insamlingsmetod, matosakerhet_p, matosakerhet_h)
    utils.populate_SKB_XYZ(SKB_XYmZ_path, SKB_XYZ_path, LG_nr, insamlingsmetod, matosakerhet_p, matosakerhet_h)
    print('SKB_XYZ populated')  

    # NEAR_DIST computed by Near3D, and then used as AvstHori in populate_SKB_XYZ,
    # does not seem to be the perpendicular distance to the wire, but rather the 
    # distance to the nearest point on the wire (due to sag, I guess)
    # So: Use NEAR analysis in 2D to create a new NEAR column, then use that one as AvstHori
    #SKB.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    utils.compute_horizontal_dist(SKB_XYZ_path, wires, search_radius)
    print('Avstand fas computed')
          
    # Now replace old avst_hori with new, correct one from compute_horizontal_dist. 
    # Then remove the NEAR-fields.
    #SKB.update_horizontal_dist(SKB_XYZ_path)
    utils.update_horizontal_dist(SKB_XYZ_path)
    print('Avstand horisontellt computed')
     
    print("Setting ground Z for XYZ points")
    all_fc = os.path.join(SKB_XYZ_path, SKB_XYZ)
    #SKB.update_z_coordinate(SKB_XYZ_location, SKB_XYZ)
    utils.update_z_coordinate(SKB_XYZ_location, SKB_XYZ)

    
          
    print(f"Done with {LG}_{line}")
# Coordinate system
sr = sweref99TM()

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(compute_SKB_distances, axis=1)
print("körning av cell klar")

# Fyll i Littera, Ursprung, Ursprung_Datum, Registreringsdatum
Borde göras i ett tidigare steg

In [ ]:
# Add data from LEDNIGNAR.txt
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

SKB_location = os.path.join(working_gdb, 'SKB_XYZ')
#print(SKB_location)

date_field = 'RGDTM'
#matosakerhet_h_field = 'MATOSAKERHET_HOJD'

urspr = "SWECO"
#ins_met = 20
#matosak_plan = 1000
#matosak_hojd = 1000
lev_dat = str(date.today())

def add_date_and_accuracy(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    #LG_code = cvd_LEDNINGSGATA[LG]
    regdatum = skanningsdatum[f"{LG}_{line}"]
    
    LG_line = f"{LG}_{line}"
    print(f"Doing {LG_line}")

    fc_SKB = f"{LG}_{line}_SKB_XYZ"
    SKB_path = os.path.join(SKB_location, fc_SKB)
    # Activating workspace
    arcpy.env.workspace = SKB_path
    arcpy.env.overwriteOutput = True
    

    
    arcpy.management.CalculateField(fc_SKB, "LITTERA", "".join(("'", littera, "'")))
    arcpy.management.CalculateField(fc_SKB, "Ursprung", "".join(("'", urspr, "'")))
    arcpy.management.CalculateField(fc_SKB, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
    arcpy.management.CalculateField(fc_SKB, "RGDTM", "".join(("'", regdatum, "'")))
    #sr = arcpy.SpatialReference("SWEREF99_TM")
    #arcpy.DefineProjection_management(fc_SKB, sr)
    # Assigning domains to fields
    arcpy.management.AssignDomainToField(
        fc_SKB, "LITTERA", "cvd_LITTERA_LEDNING")
    
    #print("Klart")

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)

df_skanningsdatum = pd.read_csv(scandate_file, sep="\t", header=0)
skanningsdatum = {f"{row['LG']}_{row['line']}": row['skanningsdatum'] for _, row in df_skanningsdatum.iterrows()}

#df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
#df_cvd_DATE = pd.read_csv(cvd_DATE_path, sep="\t", header=0)
#cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.LG[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(
#    len(df_cvd_LEDNINGSGATA))}
#cvd_DATE = {df_cvd_DATE.LG[i].join(
#    "_00" + str(df_cvd_DATE.nr[i])): df_cvd_DATE.Datum[i] for i in range(len(df_cvd_DATE))}

powerlines_df.apply(add_date_and_accuracy, axis=1)
print("körning av cell klar")

# Slå ihop kantträd för alla ledningar till en featureclass

In [ ]:
#working_gdb = "C:\SVK_2024\python_work_dir\working_250218.gdb"
print(working_gdb)
#print(Path(working_gdb,"SKB_XYZ"))
arcpy.env.workspace = working_gdb
#arcpy.env.workspace = os.path.join(working_gdb,"SKB_XYZ")
print(arcpy.env.workspace)
feature_classes = arcpy.ListFeatureClasses(feature_dataset="SKB_XYZ")
print(feature_classes)
output_feature_classes = os.path.join(working_gdb,"kantträd_oklippta") #"c:\workspace\working_25.gdb\merged_feature_class"
print(output_feature_classes)
arcpy.Merge_management(feature_classes, output_feature_classes)
print("körning av cell klar")

# Klipp träffpunkter
Se till att använda aktuella data i LG_polygons och station_polygons

In [ ]:
import arcpy
import os
import pandas as pd

#fc_ledningsgator = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_LEDNINGSGATA" #os.path.join(gdb, "Gator2021")
#fc_stationsomraden = r"Q:\Projekt\Data_2024\Underlag_SVK\Bestallningsunderlag_2024.gdb\GNG_STATIONSOMRADE" #os.path.join(gdb, "STATIONSOMRADE")

arcpy.env.overwriteOutput = True

##fd_ej_akuta = os.path.join(gdb, "ej_akuta")
#fd_all = os.path.join(gdb, "all")
#fd_traffpunkter = os.path.join(gdb, "traffpunkter")

#fc_all = os.path.join(fd_all, f"all_{version}")
#fc_traffpunkter = os.path.join(fd_traffpunkter, f"traffpunkter_{version}")
tmp_layer_traffpunkter = os.path.join(working_gdb, "tmp_layer_traffpunkter")
print(f"tmp_layer_traffpunkter: {tmp_layer_traffpunkter}")

fc_traffpunkter = os.path.join(results_gdb, 'kantträd')
print(f"fc_traffpunkter: {fc_traffpunkter}")

fc_all = os.path.join(working_gdb, "kantträd_oklippta")
print(f"fc_all: {fc_all}")

#arcpy.management.Delete(tmp_layer_traffpunkter) #behövs detta bara i fall man kör om funktionen och tmp_layer_traffpunkter existerar 



# Gör kopia av oklippta
arcpy.management.CopyFeatures(fc_all, fc_traffpunkter)

arcpy.management.MakeFeatureLayer(fc_traffpunkter, tmp_layer_traffpunkter)

# Steg 1
# Inom fastbredd och stationsområden: ta bort alla inom 7 m horisontellt avstånd från fas
#
# 1.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = LG_polygons,
                                       selection_type = "NEW_SELECTION")
# 1.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = station_polygons,
                                       selection_type = "ADD_TO_SELECTION")
# 1.3 Av de valda, behåll punkter med avst_hori < 7 m
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                        selection_type = "SUBSET_SELECTION", 
                                        where_clause = '"AVSTAND_HORISONTELLT" < 7')
# 1.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

# Steg 2
# Inom fastbredd och stationsområden: ta bort alla med trädhöjd <= 4.5 m och avst_fas >= 5 m
#
# 2.1 Välj alla inom fastbredd
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = LG_polygons,
                                       selection_type = "NEW_SELECTION")
# 2.2 Lägg till alla inom stationsområden
arcpy.management.SelectLayerByLocation(in_layer = tmp_layer_traffpunkter, 
                                       overlap_type = "INTERSECT", 
                                       select_features = station_polygons,
                                       selection_type = "ADD_TO_SELECTION")
# 2.3 Av de valda, gör gör subset selection med punkter med trädhöjd <= 4.5 OCH avst_fas >= 5
arcpy.management.SelectLayerByAttribute(in_layer_or_view = tmp_layer_traffpunkter,
                                       selection_type = "SUBSET_SELECTION",
                                       where_clause = '"DELTA_HOJD" <= 4.5 AND "AVSTAND_FAS" >= 5')
# 2.4 Radera valda
arcpy.management.DeleteFeatures(tmp_layer_traffpunkter)

## Kopiera kvarvarande punkter till traffpunkter
#arcpy.management.CopyFeatures(tmp_layer_traffpunkter, fc_traffpunkter) # Överflödigt?

# Ta bort det temporära lagret
arcpy.management.Delete(tmp_layer_traffpunkter)

print("körning av cell klar")

In [ ]:
import os
import pandas as pd
from pathlib import Path
import datetime
from datetime import date
import arcpy

#r"C:\SVK_2024\python_work_dir\results_250218.gdb"
#results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")
#print(Path(results_gdb))

arcpy.env.workspace = working_gdb
print(arcpy.env.workspace)
kanttrad = os.path.join(results_gdb, 'kantträd')
print(kanttrad)

arcpy.management.AssignDomainToField(kanttrad, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
arcpy.management.AssignDomainToField(kanttrad, "LITTERA", "cvd_LITTERA_LEDNING")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
arcpy.management.AssignDomainToField(kanttrad, "Insamlingsmetod", "cvd_INMATNINGSMETOD")



#print(kanttrad)
#arcpy.env.workspace = kanttrad
#arcpy.env.workspace(kanttrad)
#print(arcpy.env.workspace)
#feature_classes = arcpy.ListFeatureClasses(feature_dataset="SKB_XYZ")
print("körning av cell klar")


# Kontroll
### Här kan man lägga in egna skript för kontroll av resultatet

In [ ]:
# FLytta till RBX-notebook.
# Listar alla LG - littera som finns i RBX-resultatet
# Jämför detta med logg-filen för att se att rätt ledningar
# är med i resultatet (inte alla ledningar har RBX-punkter)

import arcpy
import os
import geopandas as gpd

gdb_path = r"C:\SVK_2024\pythonkörningar\results_2501xx.gdb"
feature_class = "RBX_closest_points"

gdf = gpd.read_file(gdb_path, layer=feature_class)

gdf = gdf[["LEDNINGSGATA", "LITTERA"]]
unique_df = gdf.drop_duplicates().sort_values(["LEDNINGSGATA", "LITTERA"])

print(unique_df)